## GDP Prediction  Model Evaluation

In [19]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (mean_squared_error, mean_absolute_error, 
                             r2_score, mean_absolute_percentage_error)
import matplotlib.pyplot as plt
import seaborn as sns

def create_new_features(df):
    """
    Create new features for climate and energy data analysis with robust error handling.
    """
    # Create a copy to avoid modifying the original DataFrame
    df_new = df.copy()
    
    # Define column mappings to handle potential variations
    column_map = {
        'co2_emission': ['CO2 emission (Tons)', 'CO2_emission', 'co2_emission'],
        'gdp_per_capita': ['gdp_per_capita', 'GDP per capita'],
        'population': ['Population(2022)', 'population'],
        'primary_energy_consumption': ['Primary energy consumption per capita (kWh/person)', 'primary_energy_consumption'],
        'area': ['Area(Square kilometre)', 'area'],
        'exchange_rate': ['Exchange_Rate', 'exchange_rate'],
        'gdp_growth': ['gdp_growth', 'GDP growth'],
        'renewables_percent': ['Renewables (% equivalent primary energy)', 'renewables_percent']
    }
    
    # Function to safely get column name
    def get_column(df, column_options):
        for col in column_options:
            if col in df.columns:
                return col
        raise KeyError(f"None of the columns {column_options} found in DataFrame")
    
    # Get actual column names
    co2_emission_col = get_column(df_new, column_map['co2_emission'])
    gdp_per_capita_col = get_column(df_new, column_map['gdp_per_capita'])
    population_col = get_column(df_new, column_map['population'])
    primary_energy_col = get_column(df_new, column_map['primary_energy_consumption'])
    area_col = get_column(df_new, column_map['area'])
    exchange_rate_col = get_column(df_new, column_map['exchange_rate'])
    gdp_growth_col = get_column(df_new, column_map['gdp_growth'])
    renewables_col = get_column(df_new, column_map['renewables_percent'])
    
    # Print matched columns for verification
    print("Matched Columns:")
    print(f"CO2 Emission: {co2_emission_col}")
    print(f"GDP per Capita: {gdp_per_capita_col}")
    print(f"Population: {population_col}")
    
    # Create engineered features with verified column names
    df_new['CO2_per_GDP'] = df_new[co2_emission_col] / (df_new[gdp_per_capita_col] * df_new[population_col])
    df_new['GDP_per_energy'] = df_new[gdp_per_capita_col] / df_new[primary_energy_col]
    df_new['CO2_per_area'] = df_new[co2_emission_col] / df_new[area_col]
    df_new['Real_Purchasing_Power_GDP'] = df_new[gdp_per_capita_col] / df_new[exchange_rate_col]
    
    # Group-based features
    df_new['Exchange_Rate_Volatility'] = df_new.groupby('Country')[exchange_rate_col].transform(lambda x: x.rolling(window=3).std())
    df_new['Economic_External_Sensitivity'] = df_new['Exchange_Rate_Volatility'] * df_new[gdp_per_capita_col]
    
    # Exchange-Rate Adjusted GDP Growth
    df_new['Exchange_Adjusted_GDP_Growth'] = df_new[gdp_growth_col] - df_new.groupby('Country')[exchange_rate_col].pct_change()
    
    # Time-based features
    df_new['CO2_growth_rate'] = df_new.groupby('Country')[co2_emission_col].pct_change()
    df_new['GDP_growth_per_capita'] = df_new.groupby('Country')[gdp_per_capita_col].pct_change()
    
    # CO2 Trend
    def calculate_trend(group):
        if len(group) > 1:
            first_value = group.iloc[0]
            if first_value != 0:
                return (group - first_value) / first_value
            else:
                return group - first_value
        return 0
    
    df_new['CO2_trend'] = df_new.groupby('Country')[co2_emission_col].transform(calculate_trend)
    
    # Renewable Adoption Rate
    df_new['Renewable_adoption_rate'] = df_new.groupby('Country')[renewables_col].pct_change()
    
    # Fill missing values
    columns_with_missing = df_new.columns[df_new.isna().any()].tolist()
    
    for country in df_new['Country'].unique():
        country_mask = df_new['Country'] == country
        country_data = df_new.loc[country_mask].sort_values('Year')
        
        for col in columns_with_missing:
            first_valid = country_data.loc[~country_data[col].isna(), col]
            
            if not first_valid.empty:
                first_value = first_valid.iloc[0]
                
                country_missing_mask = country_mask & df_new[col].isna()
                df_new.loc[country_missing_mask, col] = first_value
    
    return df_new

# Load original dataset and apply feature engineering
original_data_path = '/Users/chaotzuchieh/Documents/GitHub/cap5771sp25-project/Data/Final_merged_data.csv'
original_df = pd.read_csv(original_data_path)
engineered_df = create_new_features(original_df)

# Load the saved test set
test_set_path = '/Users/chaotzuchieh/Documents/GitHub/cap5771sp25-project/Data/Gdp_test_set.csv'
test_set = pd.read_csv(test_set_path)

# Load the saved model
model_path = 'best_model_gdp_growth.pkl'
best_model = joblib.load(model_path)

# Prepare model features
def get_model_features(model):
    try:
        # For pipelines or grid search models
        if hasattr(model, 'best_estimator_'):
            for step_name, step in model.best_estimator_.steps:
                if hasattr(step, 'feature_names_in_'):
                    return list(step.feature_names_in_)
        
        # For other model types
        if hasattr(model, 'feature_names_in_'):
            return list(model.feature_names_in_)
    except Exception as e:
        print(f"Error extracting features: {e}")
    
    # Fallback features based on engineered features
    return [
        'CO2_growth_rate', 'CO2_per_area', 'GDP_per_energy', 
        'Real_Purchasing_Power_GDP', 'Exchange_Rate_Volatility',
        'Economic_External_Sensitivity', 'Exchange_Adjusted_GDP_Growth'
    ]

# Get model features
model_features = get_model_features(best_model)

# Print available features
print("Available features:", list(engineered_df.columns))
print("\nModel features:", model_features)

# Verify model features exist in engineered dataframe
missing_features = [f for f in model_features if f not in engineered_df.columns]
if missing_features:
    print("\nWarning: Missing features:", missing_features)
    # Remove missing features
    model_features = [f for f in model_features if f in engineered_df.columns]

# Prepare test set with model features
X_test = engineered_df.loc[engineered_df.index.isin(test_set.index), model_features]
y_test = engineered_df.loc[engineered_df.index.isin(test_set.index), 'gdp_growth']

# Predict using the loaded model
y_pred = best_model.predict(X_test)

# Comprehensive Model Evaluation
print("\nGDP Growth Prediction Model Evaluation")
print("=" * 40)

# Regression Metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.4%}")
print(f"R-squared (R²): {r2:.4f}")

# Analyze Prediction Errors
errors = y_test - y_pred

# Distribution of Errors
plt.figure(figsize=(10, 6))
plt.hist(errors, bins=20, edgecolor='black')
plt.title('Distribution of Prediction Errors')
plt.xlabel('Prediction Error')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# Scatter Plot: Actual vs Predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Perfect Prediction')
plt.title('Actual vs Predicted GDP Growth')
plt.xlabel('Actual GDP Growth')
plt.ylabel('Predicted GDP Growth')
plt.legend()
plt.tight_layout()
plt.show()

# Errors vs Predicted Values
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, errors, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Prediction Errors vs Predicted Values')
plt.xlabel('Predicted GDP Growth')
plt.ylabel('Prediction Error')
plt.tight_layout()
plt.show()

# Top 10 Largest Prediction Errors
error_analysis = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred,
    'Absolute Error': np.abs(errors)
}).sort_values('Absolute Error', ascending=False)

print("\nTop 10 Largest Prediction Errors:")
print(error_analysis.head(10))

Matched Columns:
CO2 Emission: CO2 emission (Tons)
GDP per Capita: gdp_per_capita
Population: Population(2022)
Available features: ['Country', 'Year', 'CO2 emission (Tons)', 'Population(2022)', 'Area(Square kilometre)', 'proportion of global land area', 'Density(Square kilometre)', 'Access to electricity (% of population)', 'Access to clean fuels for cooking', 'Electricity from fossil fuels (TWh)', 'Electricity from renewables (TWh)', 'Low-carbon electricity (% electricity)', 'Primary energy consumption per capita (kWh/person)', 'Renewables (% equivalent primary energy)', 'gdp_growth', 'gdp_per_capita', 'Latitude', 'Longitude', 'Exchange_Rate', 'CO2_per_capita', 'Energy_per_CO2', 'Renewable_ratio', 'CO2_per_GDP', 'GDP_per_energy', 'CO2_per_area', 'Real_Purchasing_Power_GDP', 'Exchange_Rate_Volatility', 'Economic_External_Sensitivity', 'Exchange_Adjusted_GDP_Growth', 'CO2_growth_rate', 'GDP_growth_per_capita', 'CO2_trend', 'Renewable_adoption_rate']

Model features: ['CO2_growth_rate', 

In [ ]:


# Function for creating comprehensive visualizations
def create_comprehensive_visualizations(y_test, y_pred, features, X_test):
    """
    Create a comprehensive set of visualizations for model evaluation
    
    Parameters:
    - y_test: Actual target values
    - y_pred: Predicted target values
    - features: List of feature names
    - X_test: Feature matrix for test set
    """
    # Set up a larger figure with multiple subplots
    plt.figure(figsize=(20, 15))
    plt.suptitle('GDP Growth Prediction Model - Comprehensive Visualization', fontsize=16)

    # 1. Residual Plot (Error Distribution)
    plt.subplot(2, 3, 1)
    residuals = y_test - y_pred
    plt.scatter(y_pred, residuals, alpha=0.6, color='blue', edgecolors='black')
    plt.title('Residual Plot')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.axhline(y=0, color='r', linestyle='--')

    # 2. Q-Q Plot of Residuals
    plt.subplot(2, 3, 2)
    from scipy import stats
    stats.probplot(residuals, dist="norm", plot=plt)
    plt.title('Q-Q Plot of Residuals')

    # 3. Actual vs Predicted with Error Bands
    plt.subplot(2, 3, 3)
    plt.scatter(y_test, y_pred, alpha=0.6, color='green', edgecolors='black')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Perfect Prediction')
    
    # Calculate prediction interval (using standard error)
    std_error = np.std(residuals)
    plt.fill_between(y_test, 
                     y_pred - 1.96 * std_error, 
                     y_pred + 1.96 * std_error, 
                     alpha=0.2, color='gray')
    
    plt.title('Actual vs Predicted with 95% Prediction Interval')
    plt.xlabel('Actual GDP Growth')
    plt.ylabel('Predicted GDP Growth')

    # 4. Feature Importance Visualization
    plt.subplot(2, 3, 4)
    # Calculate feature importances (different methods depending on model type)
    try:
        # For tree-based models or models with feature_importances_
        if hasattr(best_model, 'feature_importances_'):
            importances = best_model.feature_importances_
        # For linear models
        elif hasattr(best_model, 'coef_'):
            importances = np.abs(best_model.coef_)
        else:
            importances = np.abs(np.corrcoef(X_test.T, y_test)[:-1, -1])
        
        feature_importance = pd.DataFrame({
            'feature': features,
            'importance': importances
        }).sort_values('importance', ascending=True)
        
        plt.barh(feature_importance['feature'], feature_importance['importance'])
        plt.title('Feature Importances')
        plt.xlabel('Importance')
    except Exception as e:
        plt.text(0.5, 0.5, f"Could not extract feature importances\n{str(e)}", 
                 horizontalalignment='center', verticalalignment='center')

    # 5. Kernel Density Estimation of Predictions
    plt.subplot(2, 3, 5)
    sns.kdeplot(y_test, label='Actual', fill=True, alpha=0.5)
    sns.kdeplot(y_pred, label='Predicted', fill=True, alpha=0.5)
    plt.title('Density Plot: Actual vs Predicted')
    plt.xlabel('GDP Growth')
    plt.legend()

    # 6. Box Plot of Errors
    plt.subplot(2, 3, 6)
    error_df = pd.DataFrame({
        'Actual': y_test,
        'Predicted': y_pred,
        'Absolute Error': np.abs(residuals),
        'Signed Error': residuals
    })
    sns.boxplot(data=error_df[['Absolute Error', 'Signed Error']])
    plt.title('Error Distribution')
    plt.yscale('log')

    plt.tight_layout()
    plt.show()

    # Additional Detailed Error Analysis
    print("\nDetailed Error Analysis:")
    print(f"Mean Absolute Error: {np.mean(np.abs(residuals)):.4f}")
    print(f"Median Absolute Error: {np.median(np.abs(residuals)):.4f}")
    print(f"Standard Deviation of Errors: {np.std(residuals):.4f}")
    
    # Percentile-based error analysis
    percentiles = [10, 25, 50, 75, 90]
    for p in percentiles:
        print(f"{p}th Percentile of Absolute Errors: {np.percentile(np.abs(residuals), p):.4f}")

# [Rest of the previous script remains the same, just add this function call after the existing visualizations]
create_comprehensive_visualizations(y_test, y_pred, model_features, X_test)

# Additional scatter plot matrix for feature relationships
plt.figure(figsize=(15, 15))
scatter_df = X_test.copy()
scatter_df['gdp_growth'] = y_test
scatter_matrix = pd.plotting.scatter_matrix(scatter_df, figsize=(15, 15), diagonal='kde', alpha=0.2)
plt.suptitle('Feature Relationships and Distributions', fontsize=16)
plt.tight_layout()
plt.show()


Detailed Error Analysis:
Mean Absolute Error: 2.4503
Median Absolute Error: 1.4921
Standard Deviation of Errors: 3.6590
10th Percentile of Absolute Errors: 0.1997
25th Percentile of Absolute Errors: 0.3004
50th Percentile of Absolute Errors: 1.4921
75th Percentile of Absolute Errors: 2.8920
90th Percentile of Absolute Errors: 7.0210
